In [4]:
# %pip uninstall -y torchtext
%pip install torch==1.8.0 torchtext==0.9.0
import torch
from torchtext.legacy import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm')
LABEL = data.LabelField(dtype = torch.float)

ERROR: Could not find a version that satisfies the requirement torch==1.8.0 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1)
ERROR: No matching distribution found for torch==1.8.0
Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'torchtext.legacy'

In [ ]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')
print(vars(train_data.examples[0]))

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:08<00:00, 9.73MB/s]


Number of training examples: 25000
Number of testing examples: 25000
{'text': ['How', 'you', 'could', 'say', 'that', 'Peaches', ',', 'with', 'its', 'complex', 'narrative', 'dealing', 'with', 'a', 'multitude', 'of', 'issues', ',', 'is', '"', 'a', 'small', 'TV', 'idea', '"', 'is', 'beyond', 'me', '.', 'Besides', 'I', 'can', 'think', 'of', 'many', 'films', 'that', 'have', '"', 'a', 'small', 'TV', 'idea', '"', 'in', 'their', 'plots', '.', 'Your', 'obvious', 'dislike', 'of', 'the', 'TV', 'industry', '(', '"', 'Sue', 'Smith', 'has', 'failed', 'to', 'rise', 'above', 'her', 'television', 'background', '"', ')', 'is', 'confusing', '.', 'particularly', 'as', 'you', 'are', 'having', 'such', '"', 'a', 'great', 'time', '"', 'working', 'in', 'TV', '.', 'If', 'only', 'we', 'could', 'all', 'be', 'so', 'talented', 'as', 'Ms', 'Smith', '(', 'no', ',', 'I', 'am', 'not', 'a', 'friend', 'or', 'relative', ')', '-', 'AFI', 'award', 'winning', 'Brides', 'of', 'Christ', ',', 'Road', 'from', 'Coorain', ',', 'et

In [ ]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [ ]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")
print(TEXT.vocab.freqs.most_common(20))
print(TEXT.vocab.itos[:10])
print(LABEL.vocab.stoi)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Do not use BucketIterator in your implementation because you are required to implement the padding and masking yourself.
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits((train_data, valid_data, test_data), batch_size=1, device=device)

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2
[('the', 202564), (',', 192677), ('.', 165339), ('and', 109389), ('a', 109102), ('of', 100567), ('to', 93374), ('is', 76168), ('in', 61355), ('I', 54163), ('it', 53271), ('that', 49257), ('"', 44157), ("'s", 43195), ('this', 42394), ('-', 37314), ('/><br', 35688), ('was', 35265), ('as', 30488), ('with', 30008)]
['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']
defaultdict(None, {'neg': 0, 'pos': 1})


In [ ]:
import torch.nn as nn
class LR(nn.Module):
    def __init__(self, input_dim, embedding_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.fc = nn.Linear(embedding_dim, output_dim)
    def forward(self, text):
        embedded = self.embedding(text).squeeze().sum(0)
        return self.fc(embedded)

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
OUTPUT_DIM = 1

model = LR(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,500,301 trainable parameters


In [ ]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
criterion = nn.BCEWithLogitsLoss()

In [ ]:
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
from tqdm import tqdm
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for instance in tqdm(iterator, desc="Training...", total=len(iterator)):
        
        optimizer.zero_grad()
                
        predictions = model(instance.text)
        
        loss = criterion(predictions, instance.label)
        
        acc = binary_accuracy(predictions, instance.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for instance in tqdmiterator:

            predictions = model(instance.text)
            
            loss = criterion(predictions, instance.label)
            
            acc = binary_accuracy(predictions, instance.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

100%|██████████| 17500/17500 [00:35<00:00, 491.61it/s]


Epoch: 01 | Epoch Time: 0m 41s
	Train Loss: 12.071 | Train Acc: 62.04%
	 Val. Loss: 9.109 |  Val. Acc: 62.08%


100%|██████████| 17500/17500 [00:35<00:00, 494.39it/s]


Epoch: 02 | Epoch Time: 0m 40s
	Train Loss: 8.050 | Train Acc: 65.55%
	 Val. Loss: 5.415 |  Val. Acc: 68.56%


100%|██████████| 17500/17500 [00:35<00:00, 493.77it/s]


Epoch: 03 | Epoch Time: 0m 41s
	Train Loss: 6.874 | Train Acc: 67.21%
	 Val. Loss: 6.106 |  Val. Acc: 68.64%


100%|██████████| 17500/17500 [00:35<00:00, 492.08it/s]


Epoch: 04 | Epoch Time: 0m 41s
	Train Loss: 6.504 | Train Acc: 69.42%
	 Val. Loss: 8.343 |  Val. Acc: 64.91%


100%|██████████| 17500/17500 [00:35<00:00, 493.29it/s]


Epoch: 05 | Epoch Time: 0m 41s
	Train Loss: 5.884 | Train Acc: 70.01%
	 Val. Loss: 5.660 |  Val. Acc: 71.72%


In [ ]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 5.609 | Test Acc: 67.24%


In [ ]:
# Test of model correctness
max_n_test_instances = 5
i = 1
for instance in valid_iterator:
  score = model(instance.text)
  print(score)
  if i >= max_n_test_instances:
    break
  else:
    i += 1


tensor([-6.7634], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.5967], device='cuda:0', grad_fn=<AddBackward0>)
tensor([5.0447], device='cuda:0', grad_fn=<AddBackward0>)
tensor([-0.4295], device='cuda:0', grad_fn=<AddBackward0>)
tensor([7.1855], device='cuda:0', grad_fn=<AddBackward0>)
